Coordenadas iniciales + importaciones

In [5]:
from ipyleaflet import Map, DrawControl, basemaps, basemap_to_tiles
import ipywidgets as widgets
from IPython.display import display

# Coordenadas iniciales
latitude = float(input("Introducir latitud"))
longitude = float(input("Introducir longitud"))

# Mostrar el valor introducido
print(f"Coordenadas introducidas: {latitude, longitude}")


Coordenadas introducidas: (40.08245, -4.087954)


Mapa interactivo

In [6]:
# Crear un mapa interactivo con ipyleaflet
m = Map(basemap=basemap_to_tiles(basemaps.Esri.WorldImagery), center=(latitude, longitude), zoom=15)

# Variable para almacenar las coordenadas del rectángulo
rect_coords = []

# Inicializar bounds fuera de la función
bounds = []

# Función para manejar el evento de dibujo
def handle_draw(target, action, geo_json):
    global rect_coords
    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        # Obtener las coordenadas del rectángulo
        coords = geo_json['geometry']['coordinates'][0]
        # Convertir a un formato más comprensible (SW, NE)
        latitudes = [coord[1] for coord in coords]
        longitudes = [coord[0] for coord in coords]
        rect_coords = [
            (min(latitudes), min(longitudes)),  # SW
            (max(latitudes), max(longitudes))   # NE
        ]
        print(f"Coordenadas seleccionadas: SW {rect_coords[0]}, NE {rect_coords[1]}")

# Crear un control de dibujo con opciones específicas
draw_control = DrawControl(
    rectangle={'shapeOptions': {'color': 'blue'}},
    edit=True
)
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

# Función para obtener las coordenadas del área seleccionada
def update_coords(button):
    global bounds
    if rect_coords:
        coord_label.value = f"Coordenadas seleccionadas: SW {rect_coords[0]}, NE {rect_coords[1]}"
        # Actualizar bounds con las coordenadas seleccionadas
        bounds = [rect_coords[0][0], rect_coords[0][1], rect_coords[1][0], rect_coords[1][1]]
        print(f"Bounds actualizados: {bounds}")
    else:
        coord_label.value = "No se ha seleccionado ninguna área."

# Crear un botón y etiqueta para mostrar las coordenadas
coord_label = widgets.Label()
button = widgets.Button(description="Obtener Coordenadas")
button.on_click(update_coords)

# Mostrar mapa, botón y etiqueta
display(m)
display(button)
display(coord_label)


Map(center=[40.08245, -4.087954], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

Button(description='Obtener Coordenadas', style=ButtonStyle())

Label(value='')